#`5440 Quant. ML/DatSci | SPRING 2021 | ASSIGNMENT 3 | UNI: CHB2132 `#

---

##**Problem 1 (10 points)**

**Part (a) [5 points]**

---

*Write a computer program which computes the LASSO solution path (for all λ in a given range) by implementing the cyclic coordinate descent equations derived in lecture, and the speedup due to precomputation suggested immediately thereafter in the lecture notes.*

Coordinate Descent Algorithm Process
---

Repeat until convergence or max number of iterations:

  
 1. For $i = 0,1,...,N$

 2. Compute $\rho_i = \sum_{m = 1}^M x_i^{m} (y^{m} - \hat y^{m}_{pred} + \theta_i x_i^{m} )$, 

 3. Set $\theta_i = S(\rho_i, \lambda)$

    * Note that if there is a constant term, it is not regularized ($\theta_0 = \rho_0$)

In [ ]:
#!pip install coordinatedescent

In [ ]:
import math
import copy

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from IPython.core.display import display
from IPython.core.debugger import Tracer

from datetime import datetime

# %matplotlib inline

In [ ]:
# function to compute linear regression value with lasso penalty
def lasso(x, y, betas, lambduh):
    return(np.sum(np.square(y - np.dot(x, betas))) / x.shape[0] + lambduh * np.sum(np.absolute(betas)))

In [ ]:
# function to minimize linear regression with lasso penalty: using coord. descent method

def coord_desc_cyclic(x, y, betas, lambduh, iter_max = 100):
    
    # initializing values
    iter_val = 0
    coord_val = 0
    n = x.shape[0]
    coord_max = x.shape[1]
    iter_max  = iter_max * coord_max

    # setting beta_val
    beta_val = []
    beta_val = np.zeros((iter_max, x.shape[1]), dtype = 'float')
    beta_val[0] = betas

    # looping in cyclic manner up to specified maximum iteration param
    while iter_val < iter_max:
        
        # minimizing one coordinate at a time whilst looping through them
        x_ij = pd.DataFrame(x).iloc[:, coord_val]
        a_j = 2 * np.sum(np.square(x_ij)) / n
        
        x_mj = pd.DataFrame(x).iloc[:, [j for j in range(0, x.shape[1]) if j != coord_val]]
        beta_mj = np.delete(betas, coord_val, 0)

        dot_mj = np.dot(x_mj, beta_mj)
        c_j = 2 * np.dot(np.transpose(x_ij), y - dot_mj) / n

        beta_j = coeff_calc(c_j, a_j, lambduh)
      
        betas[coord_val] = beta_j
        beta_val[iter_val] = betas

        coord_val = coord_val + 1

        if coord_val == coord_max:
            coord_val = 0
        iter_val = iter_val + 1

    return beta_val

In [ ]:
def lasso_plot(lambduh, x, y, beta_cyclic = []):

    num_points = x.shape[0]
    obj_cyclic = []
    cyclic = True
    obj_cyclic = np.zeros(num_points)

    for i in range(0, num_points):
      obj_cyclic[i] = lasso(x, y, beta_cyclic[i, :], lambduh)

    fig, ax = plt.subplots()
    ax.plot(range(1, num_points + 1), obj_cyclic, label = 'cyclic coordinate descent')

    plt.xlabel('Iteration')
    plt.title('Objective value vs. iteration when lambda = ' + str(lambduh))
    ax.legend(loc = 'upper right')
    plt.show()

    return

In [ ]:
# computes mean squared error of prediction for actual test data output

def test_MSE_calc(coeff_val, x_test_val, y_test_val):
    test_MSE = np.mean((np.dot(x_test_val, coeff_val) - y_test_val)**2)
    return test_MSE

In [ ]:
# function for coord-descent w/ cross-validation sol. (lambdas generated in log-space)

def coord_desc_CV(x, y, lambda_count = 100, iter_max = 100, fold_count = 10):

    if fold_count > x.shape[0]:
        fold_count = (int)(x.shape[0]/2)

    lambda_init = math.log10(max(abs(np.dot(x.T, y))) / x.shape[0])
 
    lambda_val = np.logspace(-8, lambda_init, lambda_count)
    lambda_val = lambda_val[np.argsort(-lambda_val)][::-1]
    lambda_val = lambda_val[::-1]

    print('cross-validation: lambda values used: ')
    print(lambda_val)

    fold_idx = range(0, x.shape[0])
    fold_idx = np.random.permutation(fold_idx)

    k_val = len(lambda_val)
    score_final = np.zeros(k_val)

    for k_iter in range(0, k_val):
        score_init = np.zeros(fold_count)

        for fold_iter in range(0, fold_count):

            x_train, x_test, y_train, y_test = split_train_test(x, y, fold_count / 100)
            beta_init = np.random.uniform(0, 0, x_train.shape[1])

            rand_cd_beta = coord_desc_cyclic(x_train, y_train, beta_init, lambda_val[k_iter], iter_max)
            score_init[fold_iter] = test_MSE_calc(rand_cd_beta[-1], x_test, y_test)

        score_final[k_iter] = np.mean(score_init)

    return lambda_val[np.argmin(score_final)]

In [ ]:
# function that splits the generated data into a training and a test dataset based on ratio
def split_train_test(x, y, p_ratio):

    train_idx = np.random.randint(0, x.shape[0], round(x.shape[0] * p_ratio))
    test_idx= np.delete(np.arange(0, x.shape[0]), train_idx)

    x_train = np.asarray(pd.DataFrame(x).iloc[train_idx,])
    y_train = np.asarray(pd.DataFrame(y).iloc[train_idx,0])

    x_test = np.asarray(pd.DataFrame(x).iloc[test_idx,])
    y_test = np.asarray(pd.DataFrame(y).iloc[test_idx,0])

    return x_train, x_test, y_train, y_test

In [ ]:
# function to calculate the coefficients

def coeff_calc(c_j, a_j, lambduh):
    if c_j < -lambduh:
        beta_j = (c_j + lambduh) / (a_j)
    elif c_j > lambduh:
        beta_j = (c_j - lambduh) / (a_j)
    else:
        beta_j = 0
    return beta_j


# **Part (b) [5 points]** 

---

*Consider a linear model with known coefficients, such as:*

$y = 3 x_1 -17 x_2 + 5 x_3 + \epsilon,$

$\epsilon \sim N(0,\sigma^2), \sigma = 1$

*From this linear model, use a random number generator to generate a simulated data set of N = 1000 pairs (X, y) according to the model. Verify that, on this simulated data set, you get the same solution path as the glmnet program by Hastie, Tibshirani and Friedman. Note that glmnet is a package you can download in R.*

*You may also use the python package discussed on Trevor Hastie's website:* https://web.stanford.edu/~hastie/glmnet_python/

In [ ]:
# function to generate sample data for specified parameters to be implemented with lasso/cv/cd funcs.

def sample_data(sample_count = 1000, feature_count = 3, standardize = False, seed_val = 1984):

  # x/input values for generated sample data
  X = feature_count
  feature_val = []
  
  # y/output values for generated sample data
  coeffs = [3, -17, 5]
  target_val = []

  # initializing random generator in numpy
  seed = seed_val
  np.random.seed(seed)
  
  # generating features using  numpy random generators
  N = sample_count
  N = (int)(math.floor(N / 3) * 3)
  sample_size = (int)(N / 3)
    
  x = pd.DataFrame([[0 for n_iter in range(N)] for x_iter in range(X)])

  for j in range(0, X):

    # first set of random method/range/scale for x value generation
    x_min = 0
    x_max = sample_size
    
    rand_loc = np.random.uniform(0, 100)
    rand_scale = np.random.uniform(275, 678)

    x.iloc[j, x_min:x_max] = np.array(np.random.normal(rand_loc, rand_scale, sample_size))

    # second set of random method/range/scale for x value generation
    x_min = sample_size
    x_max = 2 * sample_size
    
    rand_min = np.random.normal(143, 199)
    rand_max = np.random.normal(1443, 2442)

    x.iloc[j, x_min:x_max] = np.array(np.random.uniform(rand_min, rand_max, sample_size))

    # third set of random method/range/scale for x value generation
    x_min = 2 * sample_size
    x_max = 3 * sample_size
    
    rand_loc = np.random.uniform(888, 1337)
    rand_scale = np.random.uniform(789, 1234)

    x.iloc[j, x_min:x_max] = np.array(np.random.normal(rand_loc, rand_scale, sample_size))

  feature_val = np.transpose(x)
  feature_rand = feature_val + np.random.sample(X * N).reshape(N, X)

  # set coefficients as given in the linear equation for all x values
  coefficients = [3, -17, 5]

  # generate epsilon/noise
  noise = np.random.sample(N)

  # generate 'clean' y-vals' - plug rand. feature vals into given eq.
  target_clean = np.dot(feature_rand, coefficients)

  # generate final output/target vars - add noise/error (epsilon) to 'clean' y-vals
  target_val = target_clean + noise

  # using lambda function to pass standardizing function method into both X and y data as nparray objects
  if standardize == True:
      feature_val = np.asarray(pd.DataFrame(feature_val).apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x))))
      target_val = np.asarray((pd.DataFrame(target_val).apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))).iloc[:,0])

  # sets X and y data as nparray object
  feature_val = np.asarray(feature_val)
  target_val = np.asarray(target_val)

  return feature_val, target_val

In [ ]:
# generate the features for both the feature and target (x-val and y-val) for given eq./params.
feature_val, target_val = sample_data(sample_count = 1000, feature_count = 3, standardize = False, seed_val = 1984)

# split the dataset into train and test
x_train, x_test, y_train, y_test = split_train_test(feature_val, target_val, p_ratio = 0.25)                                     

max_iter = 100

beta_init = np.random.uniform(0, 0, x_train.shape[1])

# run cross-validation to get the best lambda
best_lambda_CV = coord_desc_CV(x_train, y_train, lambda_count = 100, iter_max = 100, fold_count=10)
print('Best lambda from cross validation: ' + str(best_lambda_CV))

#run the cyclic coordinate descent algorithm with the best lambda obtained from cross validation
cyclic_beta = coord_desc_cyclic(x_train, y_train, beta_init, best_lambda_CV, max_iter)

print('beta values from the final iteration')
display(cyclic_beta[-1])

#plot the objective value vs iterations
lasso_plot(best_lambda_CV, x_train, y_train, beta_cyclic = cyclic_beta)

test_MSE = test_MSE_calc(cyclic_beta[-1], x_test, y_test)

print('test mean square error: ' + str(test_MSE))


---

```
Source: https://github.com/sumanbhagavathula/cycliccoordinatedescent/
```